[View in Colaboratory](https://colab.research.google.com/github/abhiksark/kaggle_submissions/blob/master/cats_dogs_resnet.ipynb)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import files
uploaded = files.upload()

In [207]:
!mkdir /content/.kaggle
#!mv /content/.kaggle kaggle.json
#!ls
#!chmod 600 ~ kaggle.json
!mv kaggle.json /content/.kaggle
!rm -rf *
!ls /content/.kaggle/
!chmod 600 ~ /content/.kaggle/kaggle.json


mkdir: cannot create directory ‘/content/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
kaggle.json


In [208]:
!pip install kaggle

!kaggle competitions download -c dogs-vs-cats -p datalab
!ls  datalab/


sampleSubmission.csv: Downloaded 87KB of 87KB

train.zip: Downloaded 543MB of 543MB
sampleSubmission.csv  test1.zip  train.zip


In [0]:
%%capture
!unzip datalab/test1.zip
!unzip datalab/train.zip


In [0]:
%%capture
!ls
!pip install keras

In [1]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras import regularizers

Using TensorFlow backend.


In [5]:
train_dir = 'train/'
test_dir = 'test1/'
train_images_dogs_cats = [train_dir+i for i in os.listdir(train_dir)] # use this for full dataset
test_images_dogs_cats = [train_dir+i for i in os.listdir(train_dir)]

len(train_images_dogs_cats)


2

In [2]:
!mkdir train/cat/
!mkdir train/dog/
!mkdir val/
!mkdir val/cat/
!mkdir val/dog/

mkdir: cannot create directory ‘train/cat/’: File exists
mkdir: cannot create directory ‘train/dog/’: File exists
mkdir: cannot create directory ‘val/’: File exists
mkdir: cannot create directory ‘val/cat/’: File exists
mkdir: cannot create directory ‘val/dog/’: File exists


In [3]:
%%capture
src_dir = "train/"
for train_dir in train_images_dogs_cats[:20000]:
      #print(train_dir)
      train_dir_header = train_dir.split("/")
      image_header= train_dir_header[1].split(".") 
      img_class = image_header[0]
      os.rename(train_dir,src_dir + img_class + "/" + train_dir_header[1] )

NameError: ignored

In [4]:
%%capture
src_dir = "./val/"
for train_dir in train_images_dogs_cats[20000:]:
      #print(train_dir)
      train_dir_header = train_dir.split("/")
      image_header= train_dir_header[1].split(".") 
      img_class = image_header[0]
      os.rename(train_dir, src_dir + img_class + "/" + train_dir_header[1] )

      
      

NameError: ignored

In [8]:

train_dir = './train/'

datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


batch_size = 8
 
#train_features = np.zeros(shape=(nTrain, 7, 7, 512))
#train_labels = np.zeros(shape=(nTrain))
 

train_generator = datagen.flow_from_directory(
  
    train_dir,
    target_size=(200, 200),
    batch_size=batch_size,
    class_mode='binary')

Found 20000 images belonging to 2 classes.


In [9]:
val_dir = './val/'

datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
 
#train_features = np.zeros(shape=(nTrain, 7, 7, 512))
#train_labels = np.zeros(shape=(nTrain))
 

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(200,200),
    batch_size=batch_size,
    class_mode='binary')

Found 5000 images belonging to 2 classes.


In [0]:
# number of convolutional filters to use
nb_filters = 15
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 4


# number of output classes
nb_classes = 5
# number of epochs to train
nb_epoch = 64

img_rows, img_cols = 224,224

In [0]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import optimizers, Input
from keras import backend as K


In [0]:
def get_model(learn_rate=0.001, decay=0.1, trainable=False):
    """Get ResNet50 model."""
    main_model = ResNet50(include_top=False, input_tensor=Input(
        shape=(224, 224, 3)))

    main_model.trainable = trainable
    for layer in main_model.layers:
        layer.trainable = trainable

    top_model = Sequential()
    top_model.add(Flatten(input_shape=main_model.output_shape[1:]))
    # top_model.add(Dense(128, activation='relu'))
    top_model.add(Dropout(0.3))
    top_model.add(Dense(17, activation='sigmoid'))
    model = Model(inputs=main_model.input,
                  outputs=top_model(main_model.output))

    opt = optimizers.Adam(lr=learn_rate, decay=decay)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=[fbeta, 'accuracy'])

    return model


In [10]:
number_of_batches =8
counter = 0
main_model = ResNet50(include_top=False, input_tensor=Input(
        shape=(224, 224, 3)))
datagen = ImageDataGenerator(rescale=1./255,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True)

x_batch = next(datagen.flow_from_directory(
  val_dir,
  target_size=(224, 224),
  batch_size=batch_size,
  class_mode='binary'))

print(type(x_batch[1]))

Found 5000 images belonging to 2 classes.
<class 'numpy.ndarray'>


In [17]:
datagen = ImageDataGenerator(rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
process_target = True

flow_val=datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')


flow_train=datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

counter =0

def batch_generator_val(datagen,flow_val):
  number_of_batches =8
  global counter
  while True:
    counter = counter +1
    x_batch = next(flow_val,counter)
    x = main_model.predict(x_batch[0])
    y = x_batch[1]
    res_batch = (x,y)
    yield res_batch
  
def batch_generator_train(datagen,flow_train):
  number_of_batches =8  
  global counter 
  while True:
    x_batch = next(flow_train,counter)
    x = main_model.predict(x_batch[0])
    y = x_batch[1]
    res_batch = (x,y)
    yield res_batch

   
    


Found 5000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [0]:
val_generator = batch_generator_val(datagen,flow_val)
train_generator = batch_generator_train(datagen,flow_train)
  

In [20]:
next(val_generator)[0].shape

(8, 1, 1, 2048)

In [0]:
print('Training model...')
callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1),
             ModelCheckpoint(model_weights_path, monitor='val_loss',
                             save_best_only=True, verbose=0)]

steps_per_epoch_fit = np.ceil(len(fit_index) / batch_size)
steps_per_epoch_val = np.ceil(len(val_index) / batch_size)

fit_generator = batch_generator(train_dir,
                                df_train.iloc[fit_index],
                                label_map,
                                batch_size=batch_size,
                                number_of_batches=steps_per_epoch_fit,
                                data_gen_args=data_gen_args_fit,
                                seed=seed)

val_generator = batch_generator(train_dir,
                                df_train.iloc[val_index],
                                label_map,
                                batch_size=batch_size,
                                shuffle=False,
                                number_of_batches=steps_per_epoch_val,
                                data_gen_args=data_gen_args_val)
try:
    model.fit_generator(generator=fit_generator,
                        steps_per_epoch=steps_per_epoch_fit,
                        epochs=epochs,
                        verbose=1,
                        validation_data=val_generator,
                        validation_steps=steps_per_epoch_val,
                        callbacks=callbacks)

In [254]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
import keras

adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.9999, epsilon=1e-07, decay=0.00001, amsgrad=False)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_cols, img_rows, 3)))
convout1 = Activation('relu')
model.add(convout1)
model.add(Convolution2D(nb_filters, nb_conv))
model.add(MaxPooling2D(pool_size=(nb_pool+1, nb_pool+1)))
model.add(Dropout(0.6))

convout2 = Activation('relu')
model.add(convout2)
model.add(Convolution2D(nb_filters, nb_conv-2, nb_conv-2))
model.add(MaxPooling2D(pool_size=(nb_pool+2, nb_pool+2)))
model.add(Dropout(0.6))


model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.6))

model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer='adam')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, (4, 4), input_shape=(224, 224,..., padding="valid")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, (2, 2))`


In [255]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 221, 221, 15)      735       
_________________________________________________________________
activation_46 (Activation)   (None, 221, 221, 15)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 218, 218, 15)      3615      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 72, 72, 15)        0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 72, 72, 15)        0         
_________________________________________________________________
activation_47 (Activation)   (None, 72, 72, 15)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 71, 71, 15)        915       
__________

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
import keras

adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.9999, epsilon=1e-07, decay=0.00001, amsgrad=False)


model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(1,1,2048)))
model.add(Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

In [0]:
hist_plot = model.fit_generator(train_generator, steps_per_epoch=90, workers=0 ,epochs=50, validation_data=val_generator, validation_steps=30, use_multiprocessing=True)


Epoch 1/50
90/90 [==============================] - 32s 353ms/step - loss: 0.7766 - acc: 0.5056 - val_loss: 0.7073 - val_acc: 0.4583
Epoch 2/50
36/90 [===========>..................] - ETA: 13s - loss: 0.7177 - acc: 0.5833

In [0]:
0import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(hist_plot.history['acc'])
plt.plot(hist_plot.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist_plot.history['loss'])
plt.plot(hist_plot.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(hist_plot.history['acc'])
plt.plot(hist_plot.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist_plot.history['loss'])
plt.plot(hist_plot.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
#%%capture
test_images_dogs_cats = [test_dir+i for i in os.listdir(test_dir)]

x=0
prediction_probabilities = []
name = []
i=0
img_width,img_height = 200,200
for image in test_images_dogs_cats:
  x=(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
  x = np.array(x,dtype='float32')
  x = x * 1./255
  x= np.reshape(x,(1,200, 200, 3))
  print(name)
  name.append(int((image.split("/")[1]).split(".")[0]))
  prediction_probabilities.append(model.predict(x, verbose=1))
  
  print(model.predict(x, verbose=1))
  i=i+1

In [0]:
prediction_probabilities

In [0]:
solution = pd.DataFrame({"id": name, "label":prediction_probabilities})
head = solution.head()

In [0]:
solution.to_csv("dogsVScats.csv", index = False)
uploaded = drive.CreateFile({'title': 'dogsVScats.csv'})
uploaded.SetContentFile('dogsVScats.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image  =  test_images_dogs_cats[8889]
img=mpimg.imread(image)
imgplot = plt.imshow(img)
plt.show()

x=(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
print(x[0].shape)
x = np.array(x,dtype='float32')

x = x * 1./255
x= np.reshape(x,(1,200, 200, 3))

print(model.predict(x, verbose=1))


In [0]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import optimizers, Input
from keras import backend as K

In [0]:
"""Fine tuning ResNet50 with batch generator."""



def batch_generator(img_dir_path, df, label_map, batch_size=32, shuffle=True,
                    seed=2017, target_image_size=(224, 224),
                    process_target=True, number_of_batches=None,
                    add_seed_shuffle=True, data_gen_args={}, cv2_read=True,
                    preprocess_unit=False):
    """Batch generator for keras model."""
    if number_of_batches is None:
        number_of_batches = np.ceil(df.shape[0] / batch_size)
        print(number_of_batches)

    counter = 0

    if shuffle:
        np.random.seed(seed)
        df = df.sample(frac=1)

    while True:
        if process_target:
            y_batch = []

        idx_start = batch_size * counter
        idx_end = batch_size * (counter + 1)
        x_batch = []

        for f, tags in df.iloc[idx_start:idx_end].values:
            img_path = os.path.join(img_dir_path, '{}.jpg'.format(f))
            if cv2_read:
                img = cv2.imread(img_path)
                x = cv2.resize(img, target_image_size)
            else:
                img = image.load_img(img_path, target_size=target_image_size)
                x = image.img_to_array(img)

            x = np.expand_dims(x, axis=0)
            if preprocess_unit:
                x = preprocess_input(x)

            x_batch.append(x)

            if process_target:
                targets = np.zeros(17)
                for t in tags.split(' '):
                    targets[label_map[t]] = 1
                y_batch.append(targets)

        x_batch = np.concatenate(x_batch)

        datagen = ImageDataGenerator(**data_gen_args)
        datagen.fit(x_batch)
        x_batch = next(datagen.flow(x_batch, shuffle=False))

        counter += 1
        if process_target:
            yield x_batch, np.array(y_batch)
        else:
            yield x_batch

        if (counter == number_of_batches):
            if shuffle:
                if add_seed_shuffle:
                    np.random.seed(seed + 1)
                df = df.sample(frac=1)
            counter = 0